In [1]:
# standard imports
import numpy as np
import pandas as pd
from capston_db_conn import db_conn

In [2]:
conn = db_conn()
print(conn)

<connection object at 0x7fd839089d60; dsn: 'user=postgres password=xxx dbname=capstone host=mads-capstone-db.codt5pemnxi8.us-east-1.rds.amazonaws.com', closed: 0>


In [3]:
rf_sql = """
SELECT *
FROM redfin_county_full;
"""

redfin = pd.read_sql(rf_sql, con=conn, parse_dates=["period_start", "period_end"])
# conn.close()


/Users/ianbyrne/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
redfin.head()

,county_fips,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,...,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,county
0,08014,2012-04-01,2012-04-30,30,county,5,368,f,"Broomfield County, CO",None,...,0.153846,-0.20979,0.060096,1.000000,1.000000,1.000000,"Denver, CO",19740.0,2022-01-09 14:29:56,Broomfield County
1,13011,2018-05-01,2018-05-31,30,county,5,509,f,"Banks County, GA",None,...,NaN,NaN,NaN,0.111111,0.111111,-0.031746,Georgia nonmetropolitan area,NaN,2022-01-09 14:29:56,Banks County
2,39107,2019-01-01,2019-01-31,30,county,5,2213,f,"Mercer County, OH",None,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,"Celina, OH",16380.0,2022-01-09 14:29:56,Mercer County
3,39089,2018-11-01,2018-11-30,30,county,5,2204,f,"Licking County, OH",None,...,NaN,NaN,NaN,0.230769,-0.076923,-0.144231,"Columbus, OH",18140.0,2022-01-09 14:29:56,Licking County
4,44005,2013-08-01,2013-08-31,30,county,5,2430,f,"Newport County, RI",None,...,0.025547,NaN,NaN,0.279070,-0.003539,0.035827,"Providence, RI",39300.0,2022-01-09 14:29:56,Newport County


Also grab monthly unemployment by state, state income tax to add to the redfin data.

In [5]:
unem = """
SELECT date,
       rate,
       state,
       fips
FROM fred_state_monthly_ur
WHERE date >= '1/1/12'::date;
"""

unemployment = pd.read_sql(unem, con=conn, parse_dates='date')

/Users/ianbyrne/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
unemployment["year"] = unemployment["date"].dt.year
unemployment.head()

,date,rate,state,fips,year
0,2012-01-01,7.1,TX,48,2012
1,2012-02-01,7.0,TX,48,2012
2,2012-03-01,6.7,TX,48,2012
3,2012-04-01,6.8,TX,48,2012
4,2012-05-01,6.8,TX,48,2012


In [7]:
tax_q = """
SELECT sitl.state,
       sitl.state_fips,
       income_tax_low,
       income_tax_high,
       scitl.corp_income_tax_low,
       scitl.corp_income_tax_high,
       sitl.year
FROM state_income_tax_long sitl
         INNER JOIN state_corp_income_tax_long scitl
                    ON sitl.year = scitl.year AND sitl.state_fips = scitl.state_fips
WHERE sitl.year >= 2012;
"""

tax = pd.read_sql(tax_q, con=conn)

/Users/ianbyrne/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
tax.head()

,state,state_fips,income_tax_low,income_tax_high,corp_income_tax_low,corp_income_tax_high,year
0,ALABAMA,01,2.00,5.00,6.500,6.500,2012
1,ALASKA,02,0.00,0.00,1.000,9.400,2012
2,ARIZONA,04,2.59,4.54,6.968,6.968,2012
3,ARKANSAS,05,1.00,7.00,1.000,6.500,2012
4,CALIFORNIA,06,1.00,9.30,8.840,8.840,2012


In [9]:
conn.close()